In [117]:
import ipywidgets as widgets
import xarray
from io import BytesIO
import typing
import numbers
import subprocess
from shlex import quote
import requests

In [72]:
url_loc = widgets.Text(placeholder="ERDDAP NetCDF URL")
display(url_loc)

Text(value='', placeholder='ERDDAP NetCDF URL')

In [118]:
url_loc.value

'https://gliders.ioos.us/erddap/tabledap/amelia-20180501T0000.nc'

In [120]:
if uploader.value[0].type == "application/x-netcdf":
    ds = xarray.open_dataset(BytesIO(uploader.value[0].content.tobytes()))

In [125]:
url = url_loc.value
print(url)
# whether we need to store the dataset to disk before processing,
# usually due to needing to fetch a netCDF file from ERDDAP due to
# not presenting an OPeNDAP interface
store_remote = False
try:
    with warnings.catch_warnings() as w:
        warnings.simplefilter("ignore")
        ds = xarray.open_dataset(url)
        
# handle ERDDAP, which doesn't use an OPeNDAP interface for TableDAP
except OSError:
    req = requests.get(url, timeout=30)
    ds = xarray.open_dataset(BytesIO(req.content))
    store_remote = True

https://gliders.ioos.us/erddap/tabledap/amelia-20180501T0000.nc


syntax error, unexpected $end, expecting ';'
context: Error { code=404; message="Not Found: Currently unknown datasetID=amelia-20180501T0000.nc";}^


In [68]:
ds

<xarray.Dataset>
Dimensions:                                  (row: 35881)
Coordinates:
    lat_uv                                   (row) float64 ...
    lon_uv                                   (row) float64 ...
    time_uv                                  (row) datetime64[ns] ...
Dimensions without coordinates: row
Data variables: (12/64)
    trajectory                               (row) object ...
    wmo_id                                   (row) object ...
    profile_id                               (row) float64 ...
    time                                     (row) datetime64[ns] ...
    latitude                                 (row) float64 ...
    longitude                                (row) float64 ...
    ...                                       ...
    time_qc                                  (row) float32 ...
    time_uv_qc                               (row) float32 ...
    u                                        (row) float64 ...
    u_qc                                     (row) float32 ...
    v                                        (row) float64 ...
    v_qc                                     (row) float32 ...
Attributes: (12/56)
    acknowledgment:                This deployment supported by Heising-Simon...
    cdm_data_type:                 TrajectoryProfile
    cdm_profile_variables:         time_uv,lat_uv,lon_uv,u,v,profile_id,time,...
    cdm_trajectory_variables:      trajectory,wmo_id
    contributor_name:              Donglai Gong, Jeanna Hudson, Lydia Bienlie...
    contributor_role:              Principal Investigator, Glider Pilot, Glid...
    ...                            ...
    subsetVariables:               wmo_id,trajectory,profile_id,time,latitude...
    summary:                       This is a test mission for the Northwest P...
    time_coverage_end:             2018-05-14T12:29:12Z
    time_coverage_start:           2018-05-01T14:58:48Z
    title:                         amelia-20180501T0000
    Westernmost_Easting:           -75.07957909499224

In [126]:
import ipywidgets as widgets

time_variable_selection = widgets.Dropdown(
    #options=['1', '2', '3'],
    options=[k for k in ds.variables.keys()],
    #value='2',
    description='Time variable:',
    disabled=False,
)
#time_variable = ds.variables[time_selection.value]
display(time_variable_selection)

Dropdown(description='Time variable:', options=('trajectory', 'wmo_id', 'profile_id', 'time', 'latitude', 'lon…

In [127]:
input_variable_selection = widgets.SelectMultiple(
    #options=['1', '2', '3'],
    options=[k for k in ds.variables.keys()],
    #value='2',
    description='Input variable(s):',
    disabled=False,
)
display(input_variable_selection)

SelectMultiple(description='Input variable(s):', options=('trajectory', 'wmo_id', 'profile_id', 'time', 'latit…

In [77]:
x_variable_selection = widgets.Dropdown(
    options=[k for k in ds.variables.keys()],
    description='x/lon variable:',
    disabled=False,
)
display(x_variable_selection)

Dropdown(description='x/lon variable:', options=('trajectory', 'wmo_id', 'profile_id', 'time', 'latitude', 'lo…

In [78]:
y_variable_selection = widgets.Dropdown(
    options=[k for k in ds.variables.keys()],
    description='y/lat variable:',
    disabled=False,
)
display(y_variable_selection)

Dropdown(description='y/lat variable:', options=('trajectory', 'wmo_id', 'profile_id', 'time', 'latitude', 'lo…

In [37]:
depth_variable_selection = widgets.Dropdown(
    options=[None] + [k for k in ds.variables.keys()],
    description='Depth/z variable:',
    disabled=False,
)
display(depth_variable_selection)

Dropdown(description='Depth/z variable:', options=(None, 'trajectory', 'wmo_id', 'profile_id', 'time', 'latitu…

In [129]:
time_variable = ds.variables[time_selection.value]
input_variable_dict = {var_name: ds.variables[var_name] for var_name in input_variable_selection.value}
x_variable = ds.variables[x_variable_selection.value]
y_variable = ds.variables[y_variable_selection.value]
depth_variable = None if depth_variable_selection.value is None else ds.variables[depth_variable_selection.value]

In [130]:
from ioos_qc import qartod

In [132]:
import inflection
import warnings
import numbers


def widget_factory_type(variable_name, function):
    widget_controls = []
    annotations = function.__annotations__
    # TODO: use proper map widget for selecting location test watch bounds
    #bbox_type = typing.Tuple[numbers.Real, numbers.Real, numbers.Real, numbers.Real]
    for annotation_name, type_signature in annotations.items():
        # data variables should be skipped as they aren't tunable for QARTOD test parameters
        if annotation_name in {"inp", "tinp", "lon", "lat", "zinp", "method", "return", "type_signature", "check_type"}:
            continue
            
        match type_signature:
            case t if t is numbers.Real or t is float:
                widget_controls.append(widgets.FloatSlider(value=0, min=-500, max=500,
                                                           description=annotation_name,
                                                           name=f"{variable_name} {function} {annotation_name}"))
            case t if t is int:
                widget_controls.append(widgets.IntSlider(value=0, min=0, max=100,
                                                    description=annotation_name,
                                                    name=f"{variable_name} {function} {annotation_name}"))
            case t if t is typing.Tuple[numbers.Real, numbers.Real]:
                widget_controls.append(widgets.FloatRangeSlider(value=[-100, 100], min=-500, max=500,
                                                                description=annotation_name,
                                                                name=f"{variable_name} {function} {annotation_name}"))             
            # four numbers could only be geospatial bounding box for location test
            # TODO: display with interact, etc
            case t if t is typing.Tuple[numbers.Real, numbers.Real, numbers.Real, numbers.Real]:
                widget_controls.append(widgets.VBox([widgets.FloatRangeSlider(value=[0, 1], min=-180, max=180,
                                                           description="Longitude Bounds"),
                                  widgets.FloatRangeSlider(value=[0, 1], min=-90, max=90,
                                                           description="Latitude Bounds")],
                                                    name=f"{variable_name} {function} {annotation_name}"))
            case _:
                warnings.warn(f"Parameter {annotation_name} with type signature {type_signature} unmatched, skipping...")
                continue
    checkbox = widgets.Checkbox(False, description="Disabled", indent=False)
    for control in widget_controls:
        widgets.link((checkbox, 'value'), (control, 'disabled'))
    return widgets.VBox([checkbox, widgets.VBox(widget_controls)])

In [133]:
def show_controls(variable_name, function): 
    print(inflection.titleize(function.__name__))
    display(widget_factory_type(variable_name, function))

In [134]:
import inspect
from ioos_qc import qartod

test_dict = {}
qartod_tests = inspect.getmembers(qartod, lambda o: inspect.isfunction(o) and o.__name__.endswith("_test"))
for input_variable_name, input_variable in input_variable_dict.items():
    test_dict[input_variable_name] = {}
    print(input_variable_name)
    for test_name, qartod_test in qartod_tests:
        # don't know how to represent climatology widgets
        if test_name in {"climatology_test", "location_test"}:
            continue
        #print(qartod_test)
        test_dict[input_variable_name][test_name] = widget_factory_type(input_variable_name, qartod_test)
        print(inflection.titleize(qartod_test.__name__))
        show_controls(input_variable_name, qartod_test)
display(widgets.Checkbox(True, description="Run CF Compliance Check", indent=False))

temperature
Attenuated Signal Test
Attenuated Signal Test


Density Inversion Test
Density Inversion Test


Flat Line Test
Flat Line Test


Gross Range Test
Gross Range Test


Rate Of Change Test
Rate Of Change Test


Spike Test
Spike Test


Checkbox(value=True, description='Run CF Compliance Check', indent=False)

Button(description='Submit', style=ButtonStyle())

In [135]:
def generate_qc_json_config(widget_conf):
    qc_conf_dict = {}
    for variable_name in widget_conf:
        qc_conf_dict[variable_name] = {}
        qartod_conf = qc_conf_dict[variable_name]["qartod"] = {}
        for test_name in widget_conf[variable_name]:
            # if test is disabled, skip
            if test_dict[variable_name][test_name].children[0].value:
                continue
            qartod_conf[test_name] = {}
            test_widgets = test_dict[variable_name][test_name].children[1].children
            for test_widget in test_widgets:
                qartod_conf[test_name][test_widget.description] = test_widget.value
    return qc_conf_dict

qc_config = generate_qc_json_config(test_dict)
            

In [162]:
cmds = ["argo", "submit", "-v",
                "-p", f"file-url={quote(url)}",
                "-p", f"qc-config={quote(json.dumps(qc_config))}",
                "-p", f"input-var-selection={quote(json.dumps(input_variable_selection.value))}",
                "-p", f"x-var-name={quote(x_variable_selection.value)}",
                "-p", f"y-var-name={quote(y_variable_selection.value)}",
                "-p", f"z-var-name={quote(depth_variable_selection.value)}",
                "-p", f"t-var-name={quote(time_variable_selection.value)}", 
                "argo_workflows/curl_file.yaml"]

subprocess.run(cmds)

time="2023-03-17T18:52:58.097Z" level=debug msg="CLI version" version="{v3.4.3 2022-10-31T09:13:13Z eddb1b78407adc72c08b4ed6be8f52f2a1f1316a v3.4.3 clean go1.18.7 gc linux/amd64}"
time="2023-03-17T18:52:58.097Z" level=debug msg="Client options" opts="(argoServerOpts=(url=,path=,secure=true,insecureSkipVerify=false,http=false),instanceID=)"
I0317 18:52:58.097920    2790 loader.go:372] Config loaded from file:  /home/badams/.kube/config
I0317 18:52:58.098912    2790 loader.go:372] Config loaded from file:  /home/badams/.kube/config
time="2023-03-17T18:52:58.104Z" level=debug msg="Getting the template by name"
time="2023-03-17T18:52:58.104Z" level=debug msg="Resolving the template" base="*v1alpha1.Workflow (namespace=default,name=)" depth=0 tmpl="*v1alpha1.WorkflowStep (curl-file-and-process)"
time="2023-03-17T18:52:58.104Z" level=debug msg="Getting the template" base="*v1alpha1.Workflow (namespace=default,name=)" depth=0 tmpl="*v1alpha1.WorkflowStep (curl-file-and-process)"
time="2023-03

Name:                curl-to-artifact-qwz6v
Namespace:           default
ServiceAccount:      unset (will run with the default ServiceAccount)
Status:              Pending
Created:             Fri Mar 17 18:52:59 -0400 (now)
Progress:            
Parameters:          
  file-url:          https://gliders.ioos.us/erddap/tabledap/amelia-20180501T0000.nc
  qc-config:         '{"temperature": {"qartod": {"attenuated_signal_test": {"suspect_threshold": 0.0, "fail_threshold": 0.0, "test_period": 0.0, "min_obs": 0.0, "min_period": 0}, "density_inversion_test": {"suspect_threshold": 0.0, "fail_threshold": 0.0}, "flat_line_test": {"suspect_threshold": 0, "fail_threshold": 0, "tolerance": 0.0}, "gross_range_test": {"fail_span": [-100.0, 100.0], "suspect_span": [-100.0, 100.0]}, "rate_of_change_test": {"threshold": 0.0}, "spike_test": {"suspect_threshold": 0.0, "fail_threshold": 0.0}}}}'
  input-var-selection: '["temperature"]'
  x-var-name:        longitude
  y-var-name:        latitude
  z-var-

I0317 18:52:59.306134    2790 round_trippers.go:553] POST https://47B3AF6215C3C908448CBFA0FFF755DA.gr7.us-east-1.eks.amazonaws.com/apis/argoproj.io/v1alpha1/namespaces/default/workflows 201 Created in 1200 milliseconds


CompletedProcess(args=['argo', 'submit', '-v', '-p', 'file-url=https://gliders.ioos.us/erddap/tabledap/amelia-20180501T0000.nc', '-p', 'qc-config=\'{"temperature": {"qartod": {"attenuated_signal_test": {"suspect_threshold": 0.0, "fail_threshold": 0.0, "test_period": 0.0, "min_obs": 0.0, "min_period": 0}, "density_inversion_test": {"suspect_threshold": 0.0, "fail_threshold": 0.0}, "flat_line_test": {"suspect_threshold": 0, "fail_threshold": 0, "tolerance": 0.0}, "gross_range_test": {"fail_span": [-100.0, 100.0], "suspect_span": [-100.0, 100.0]}, "rate_of_change_test": {"threshold": 0.0}, "spike_test": {"suspect_threshold": 0.0, "fail_threshold": 0.0}}}}\'', '-p', 'input-var-selection=\'["temperature"]\'', '-p', 'x-var-name=longitude', '-p', 'y-var-name=latitude', '-p', 'z-var-name=depth', '-p', 't-var-name=time', 'argo_workflows/curl_file.yaml'], returncode=0)